In [1]:
from __future__ import division, print_function
%load_ext autoreload
%autoreload 2
%pylab inline
# %pylab notebook
# %load_ext wurlitzer
# standard imports

import time
import numpy as np
from collections import namedtuple
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# drake + externals imports
from director import viewerclient as vc
from irispy import Polyhedron
import pydrake.solvers.mathematicalprogram as mathematicalprogram

# boxatlas
import boxatlas.boxatlas as box
from boxatlas.contactstabilization import BoxAtlasContactStabilization
from boxatlas.contactstabilizationutils import ContactStabilizationUtils as CSU
from utils.polynomial import Polynomial
from utils.piecewise import Piecewise

Populating the interactive namespace from numpy and matplotlib


In [2]:
# construct a visualizer object
vis = vc.Visualizer()

In [3]:
# setup and solve a standard box atlas contact stabilization problem
# Meant to be an example and be kept as simple as possible
env = CSU.make_environment()
robot = CSU.make_box_atlas()

desired_state = CSU.make_default_desired_state(robot)

# make an initial state
initial_state = CSU.make_default_initial_state(robot)
initial_state.vcom[0] = -1.75 # set initial velocity of com
initial_state.contact_indicator = np.array([0,1,1,0], dtype=int)

num_time_steps = 20
dt = 0.05

# constrain some limbs to either be in contact or not
constrained_limbs = dict()
constrained_limbs["right_leg"] = 1 # persistent contact
constrained_limbs["left_leg"] = 1 # persistent contact
# constrained_limbs["right_arm"] = 0 # no contact

contact_assignments = CSU.make_contact_assignment(dt, num_time_steps,
                                              constrained_limbs=constrained_limbs)

# get optimization parameters
params = CSU.get_default_optimization_parameters()

options = dict()
options["use_lambda_contact_formulation"] = False

opt = BoxAtlasContactStabilization(robot, initial_state, env, desired_state,
                             num_time_steps=num_time_steps,
                             dt=dt,
                             params=params,
                             contact_assignments=contact_assignments, options=options)


solnData = opt.solve()
box.planPlayback(vis, solnData, slider=True)
print("solution took = ", solnData.solve_time)
CSU.plot_contact_indicator(solnData)

None